# Use Cases for VIGOR

This notebook is to demonstrate a use case for VIGOR

In [1]:
import csv
from neo4j import GraphDatabase
from neo4j.graph import Node, Relationship
from vigor import Graph, VIGOR, predicates, Visualization, Predicate

## Recommendation Use Case

Uses a sample database provided by Neo4j

In [2]:
vigor = VIGOR()

visualization_preds = {}
for vis, score, stat, min, max in predicates:
    if vis not in visualization_preds:
        visualization_preds[vis] = Visualization(vis)
    visualization_preds[vis].add_predicate(Predicate(stat, min, max, score))

for vis in visualization_preds.values():
    vigor.add_visualization(vis)

In [3]:
queries = {
    "recommendations": [
      {
        "task": "Topology-Based",
        "question": "Which director directed the most amount of movies?",
        "query": "MATCH (d:Director)-[r:DIRECTED]-(m:Movie) RETURN d, r, m"
      },
      {
        "task": "Attribute-Based",
        "question": "Which of the movies has the heighest average rating by a user?",
        "query": "MATCH (u:User)-[r:RATED]-(m:Movie) RETURN u, r, m"
      },
      {
        "task": "Browsing",
        "question": "Can you find all actors that also directed the movie they played in?",
        "query": "MATCH (a:Actor)-[r:ACTED_IN]->(m:Movie)-[d:DIRECTED]-(e:Actor) RETURN a, r, m, d, e"
      },
       {
        "task": "Overview",
        "question": "Can you estimate the size of the entire network?",
        "query": "MATCH * RETURN *"
      }
    ],
}

In [4]:
NEO4J_CONNECTION_URI="neo4j+s://demo.neo4jlabs.com:7687"
csv_filename = "../data/use-cases.csv"

In [ ]:
header_written = False

for database in queries:
    driver = GraphDatabase.driver(NEO4J_CONNECTION_URI, auth=(database, database))
    for query in queries[database]:
        G = Graph()

        records, summary, keys = driver.execute_query(query['query'], database_=database)
        print('ececuted query')
        for record in records:
            for element in record:
                if isinstance(element, Node):
                    G.add_node(record[0], label=element.labels, attributes=element.items())
                elif isinstance(element, Relationship):
                    G.add_edge(record[0], record[1], type=element.type, attributes=element.items())

        print('created graph')
        graph_stats = G.get_statistics()
        print(graph_stats)

        # Unpack recommendations and scores
        recommendations_with_scores = vigor.recommend(graph_stats, 8)
        recommendations = [rec[0].value for rec in recommendations_with_scores]  # Extract recommendations
        scores = [rec[1] for rec in recommendations_with_scores]  # Extract scores

        # print("Recommended visualizations:", recommendations)

        # Add recommendations and scores to graph stats
        graph_stats['database_name'] = database
        graph_stats['question'] = query['question']
        graph_stats['query'] = query['query']
        graph_stats['q_type'] = query['q_type']
        graph_stats["rec_1"] = recommendations[0]
        graph_stats["rec_2"] = recommendations[1]
        graph_stats["rec_3"] = recommendations[2]
        graph_stats["score_1"] = scores[0]
        graph_stats["score_2"] = scores[1]
        graph_stats["score_3"] = scores[2]

        # Open the CSV file in append mode
        with open(csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)

            if not header_written:
                writer.writerow(graph_stats.keys())
                header_written = True

            writer.writerow(graph_stats.values())

ececuted query
created graph


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1044da8e0>>
Traceback (most recent call last):
  File "/Users/sjoerdvink/Developer/Experiments/vigor-visualization-recommendation-for-graphs/.venv/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
